In [24]:
# Setup imports and path
import sys
import os
import uuid
sys.path.append('..')  # Add parent directory to path

from core.dst_manager import DSTManager
from dotenv import load_dotenv
from loaders.node_loader import load_nodes
import logging

# Load environment variables
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logging.getLogger('openai._base_client').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)

print("Imports successful!")


Imports successful!


In [25]:
# Load chatbot configuration and initialize DST Manager
config_path = '../config/card_issuance_chatbot.json'

try:
    # Load nodes from configuration
    nodes = load_nodes(config_path, enable_graph_validation=False)
    print(f"Loaded {len(nodes)} nodes from config")
    
    # Initialize DST Manager with the loaded nodes
    dst_manager = DSTManager(
        nodes_config=nodes, 
        use_redis=False,
        start_node=None,  # Auto-detect start node
        enable_llm_stage_detection=True
    )
    
    print("DST Manager initialized successfully!")
    print(f"Available nodes: {list(nodes.keys())}")
    print(f"Start node: {dst_manager.start_node}")
    
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


INFO:loaders.node_loader:Loaded 9 nodes from ../config/card_issuance_chatbot.json
INFO:storage.context_store:Using in-memory storage for context
INFO:core.dst_manager:DST Manager initialized with 9 nodes
INFO:core.dst_manager:Start node: welcome
INFO:core.dst_manager:Stage detection: enabled


 로드된 노드 수: 9
Loaded 9 nodes from config
9개 노드 추가 완료
✅ 그래프 생성 완료!
노드 수: 9
엣지 수: 8

✅ 순환 없음 - 그래프가 DAG입니다.
위상 정렬 순서: welcome -> collect_personal_info -> select_card_type -> income_verification -> address_collection -> final_confirmation -> process_application -> completion_approved -> completion_rejected
DST Manager initialized successfully!
Available nodes: ['welcome', 'collect_personal_info', 'select_card_type', 'income_verification', 'address_collection', 'final_confirmation', 'process_application', 'completion_approved', 'completion_rejected']
Start node: welcome


In [26]:
# Start a new session
session_id = str(uuid.uuid4())
print(f"Starting new session: {session_id}")

try:
    session_id = dst_manager.start_session(session_id)
    print("Session started successfully!")
    
    # Get session info
    session_info = dst_manager.get_session_info(session_id)
    if session_info:
        print(f"Current node: {session_info.get('current_node')}")
        print(f"Turn count: {session_info.get('turn_count', 0)}")
        
except Exception as e:
    print(f"Failed to start session: {e}")
    import traceback
    traceback.print_exc()

print("Ready for chatbot interactions!")


INFO:core.dst_manager:세션 시작: d17fd83f-2bf0-4071-b34c-28f621398083


Starting new session: d17fd83f-2bf0-4071-b34c-28f621398083
Session started successfully!
Current node: welcome
Turn count: 0
Ready for chatbot interactions!


In [27]:
# Test chatbot interaction
def test_interaction(user_message, show_debug=True):
    """Test a single interaction with the chatbot"""
    try:
        print(f"\nUser: {user_message}")
        
        # Process user message
        result = dst_manager.process_turn(session_id, user_message)
        
        # Display response
        print(f"Bot: {result.get('response', 'No response')}")
        
        if show_debug:
            print(f"Debug - Current node: {result.get('current_node', 'Unknown')}")
            print(f"Debug - Turn count: {result.get('turn_count', 0)}")
            print(f"Debug - Session complete: {result.get('session_complete', False)}")
            
            slots = result.get('slots', {})
            if slots:
                print("Debug - Collected slots:")
                for slot_name, slot_value in slots.items():
                    print(f"  {slot_name}: {slot_value}")
        
        return result
        
    except Exception as e:
        print(f"Error in interaction: {e}")
        import traceback
        traceback.print_exc()
        return None

def reset_session():
    """Reset the current session"""
    try:
        dst_manager.reset_session(session_id)
        print("Session reset successfully.")
    except Exception as e:
        print(f"Failed to reset session: {e}")

print("Test functions defined!")


Test functions defined!


In [28]:
# Test the chatbot with a greeting
test_interaction("안녕하세요")



User: 안녕하세요
Bot: 안녕하세요! 마고은행 카드 발급 서비스에 오신 것을 환영합니다. 😊 평일 오전 9시부터 오후 6시까지 서비스를 제공하고 있으니, 궁금한 점이나 도움이 필요하시면 언제든 말씀해주세요! 카드 발급을 시작해볼까요?
Debug - Current node: collect_personal_info
Debug - Turn count: 1
Debug - Session complete: False


{'response': '안녕하세요! 마고은행 카드 발급 서비스에 오신 것을 환영합니다. 😊 평일 오전 9시부터 오후 6시까지 서비스를 제공하고 있으니, 궁금한 점이나 도움이 필요하시면 언제든 말씀해주세요! 카드 발급을 시작해볼까요?',
 'session_id': 'd17fd83f-2bf0-4071-b34c-28f621398083',
 'current_node': 'collect_personal_info',
 'turn_count': 1,
 'session_complete': False,
 'slots': {},
 'context': {'last_intent': 'greeting',
  'last_entities': {},
  'last_confidence': 0.95,
  'start': True,
  'greeted': True,
  'last_user_message': '안녕하세요'}}

In [29]:
# Continue the conversation - modify the message below as needed
user_message = "안녕하세요"  # Change this message for different tests

test_interaction(user_message)



User: 안녕하세요
Bot: 안녕하세요! 신용카드 발급을 위해 몇 가지 정보가 필요합니다. 먼저 성함과 생년월일(예: 900101)을 알려주실 수 있을까요? 😊
Debug - Current node: collect_personal_info
Debug - Turn count: 2
Debug - Session complete: False


{'response': '안녕하세요! 신용카드 발급을 위해 몇 가지 정보가 필요합니다. 먼저 성함과 생년월일(예: 900101)을 알려주실 수 있을까요? 😊',
 'session_id': 'd17fd83f-2bf0-4071-b34c-28f621398083',
 'current_node': 'collect_personal_info',
 'turn_count': 2,
 'session_complete': False,
 'slots': {},
 'context': {'last_intent': 'greeting',
  'last_entities': {},
  'last_confidence': 0.95,
  'start': True,
  'greeted': True,
  'last_user_message': '안녕하세요'}}

In [30]:
# Check current session status
session_info = dst_manager.get_session_info(session_id)

if session_info:
    print("Current session status:")
    print(f"  Session ID: {session_id}")
    print(f"  Current node: {session_info.get('current_node')}")
    print(f"  Turn count: {session_info.get('turn_count', 0)}")
    print(f"  Complete: {'Yes' if session_info.get('is_complete') else 'No'}")
    
    slots = session_info.get('slots', {})
    if slots:
        print("  Collected information:")
        for slot_name, slot_value in slots.items():
            print(f"    {slot_name}: {slot_value}")
    else:
        print("  No information collected yet")
else:
    print("Session information not found")


Current session status:
  Session ID: d17fd83f-2bf0-4071-b34c-28f621398083
  Current node: collect_personal_info
  Turn count: 2
  Complete: No
  No information collected yet
